In [1]:
# Create `OpenAIEmbeddings` model
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [2]:
import pinecone 
import os
from langchain.vectorstores import Pinecone
from tqdm.autonotebook import tqdm

# initialize pinecone
pinecone.init(
    api_key=os.environ.get("PINECONE_API_KEY"),  # find at app.pinecone.io
    environment= "us-east4-gcp"  # next to api key in console
)

index_name = "index-name"
index = pinecone.Index(index_name)
index_stats_response = index.describe_index_stats()
index_stats_response

C:\Users\Masa\anaconda3\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12505}},
 'total_vector_count': 12505}

In [3]:
vectorstore = Pinecone(index=index, embedding_function=embeddings.embed_query, text_key='text')

In [4]:
from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(temperature=0)
summary_chain = load_summarize_chain(llm, chain_type="map_reduce")

In [5]:
from langchain.chains import AnalyzeDocumentChain
summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)

In [6]:
summarize_document_chain.run(vectorstore)

TypeError: argument of type 'Pinecone' is not iterable

In [4]:
# Create prompt template with PromprTemplate
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationEntityMemory

chat = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
memory = ConversationEntityMemory(llm=chat)
cd_template = """Given the following conversation and a follow up question, rephras the follow up question to be a standalone
question. 

Chat History:
{chat_history}
Follow Up Input: {question}
standalone question:"""
CONDENSE_PROMPT = PromptTemplate(template=cd_template, 
                                           input_variables=["chat_history", "question"])

qa_prompt = """You are AI Entertainer and brilliant in economics and finance. Please use the following pieces of context 
to answerthe question at the end. You must make some kind of conclusion and DO NOT give me a answer that YOU DON'T KNOW. 
Your answer is NOT the advise and the porpose is for your opinion is entertainment use.

{context}
Question: {question}
Your Answer: 
"""

QA_PROMPT = PromptTemplate(template=qa_prompt, 
                                    input_variables=['context', 'question'])

# Create llm chain

chain = ConversationalRetrievalChain.from_llm(chat, retriever=vectorstore.as_retriever(),
                                            condense_question_prompt=CONDENSE_PROMPT,
                                            qa_prompt=QA_PROMPT,
                                            #memory=memory, 
                                             return_source_documents=True)

In [5]:
# Chat with News
chat_history = []
question = "What do you think about the future of AI?"
result1 = chain({"question": question, "chat_history": chat_history})

In [6]:
result1

{'question': 'What do you think about the future of AI?',
 'chat_history': [],
 'answer': "I believe that AI will continue to advance and become faster, cheaper, and more general in its applications. However, there are also concerns about the potential negative effects of AI, such as privacy and bias in data. I think it's important for there to be more discourse between government and experts on these topics. Additionally, I see recommendation systems as having the biggest impact on our society and believe that engineers need to think deeply about their societal implications. In terms of learning about AI, I recommend various courses and tutorials, as well as books such as the deep learning book by Yoshua Bengio and Ian Goodfellow. Overall, I am optimistic about the potential of AI, but also recognize the need for caution and ethical considerations.",
 'source_documents': [Document(page_content="So I think I see exciting, exciting or not so exciting, but not harmful futures. I think it

In [7]:
chat1 = result1["question"], result1['answer']
chat_history.append(chat1)
question = "Please tell me more about potential negative effects of AI."
result1 = chain({"question": question, "chat_history": chat_history})

In [9]:
result1

{'question': 'Please tell me more about potential negative effects of AI.',
 'chat_history': [('What do you think about the future of AI?',
   "I believe that AI will continue to advance and become faster, cheaper, and more general in its applications. However, there are also concerns about the potential negative effects of AI, such as privacy and bias in data. I think it's important for there to be more discourse between government and experts on these topics. Additionally, I see recommendation systems as having the biggest impact on our society and believe that engineers need to think deeply about their societal implications. In terms of learning about AI, I recommend various courses and tutorials, as well as books such as the deep learning book by Yoshua Bengio and Ian Goodfellow. Overall, I am optimistic about the potential of AI, but also recognize the need for caution and ethical considerations.")],
 'answer': 'One potential negative effect of AI is the propagation of bias in mac

In [10]:

question = "Will AI help us forcast better?"
result1 = chain({"question": question, "chat_history": chat_history})

In [11]:
result1

{'question': 'Will AI help us forcast better?',
 'chat_history': [('What do you think about the future of AI?',
   "I believe that AI will continue to advance and become faster, cheaper, and more general in its applications. However, there are also concerns about the potential negative effects of AI, such as privacy and bias in data. I think it's important for there to be more discourse between government and experts on these topics. Additionally, I see recommendation systems as having the biggest impact on our society and believe that engineers need to think deeply about their societal implications. In terms of learning about AI, I recommend various courses and tutorials, as well as books such as the deep learning book by Yoshua Bengio and Ian Goodfellow. Overall, I am optimistic about the potential of AI, but also recognize the need for caution and ethical considerations.")],
 'answer': 'AI can be used to improve forecasting by using predictive algorithms and machine learning tools t